# ZSL modification of original training code

This code should be used for ZSL only, not for the generalized ZSL scenario.


In [1]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

from zsldataset import ZSLDataset
from models import ContinuousMap, ContinuousMapResidual

def dist_matrix(batch1, batch2):
    delta = batch1.unsqueeze(1) - batch2.unsqueeze(0)
    
    dist_matrix = (delta * delta).mean(dim=-1)
    
    return dist_matrix


def mag(u):
    return torch.dot(u, u)


def dist(u, v):
    return torch.dot(u - v, u - v)

## Load AWA dataset

Should be converted into ZSL format beforehand.

In [2]:
trainset = ZSLDataset('Data/AWA/train', use_predicates=False, use_irevnet=True)
testset = ZSLDataset('Data/AWA/test', use_predicates=False, use_irevnet=True)

In [3]:
bsize = 128
n_epochs = 200
num_classes = trainset.classes.shape[0]

dim_semantic = trainset[0]['class_embedding'].shape[0]
dim_visual = trainset[0]['image_embedding'].shape[0]

all_class_embeddings = torch.tensor(np.array(trainset.class_embeddings)).cuda().float()
classes_enum = torch.tensor(np.array(range(num_classes), dtype=np.int64)).cuda()

In [4]:
query_classes = set([testset[i]['class_label'] for i in range(len(testset))])
query_ids = set([testset[i]['class_id'] for i in range(len(testset))])

In [5]:
ids = list(i-1 for i in query_ids)
query_mask = np.zeros((num_classes))
query_mask[ids] = 1
query_mask = torch.tensor(query_mask, dtype=torch.int64).cuda()

In [11]:
v_to_s = ContinuousMap(dim_source=dim_visual, dim_dest=dim_semantic, width=512).cuda()
s_to_v = ContinuousMap(dim_source=dim_semantic, dim_dest=dim_visual, width=512).cuda()

# optimizer = torch.optim.Adam(list(v_to_s.parameters()) + list(s_to_v.parameters()),
#                                 lr = 1e-3,
#                                 betas=(0.9, 0.999),
#                                 weight_decay=3e-2)

optimizer = torch.optim.SGD(list(v_to_s.parameters()) + list(s_to_v.parameters()),
                                lr = 1e-4,
                                momentum=.9,
                                weight_decay=3e-2)

In [12]:
mse = torch.nn.MSELoss()
triplet_loss = torch.nn.TripletMarginLoss(margin=1, p=2)
positive_part = torch.nn.ReLU()

In [13]:
trainloader = torch.utils.data.DataLoader(trainset, 
                                             batch_size=bsize, 
                                             shuffle=True, 
                                             num_workers=4, 
                                             pin_memory=True, 
                                             drop_last=True)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=bsize, 
                                         shuffle=True, 
                                         num_workers=4, 
                                         pin_memory=True, 
                                         drop_last=True)

In [16]:
alpha1 = 10 # triplet
alpha2 = 1e-3 # surjection
alpha3 = 1e-3 # l2 loss

margin = 1

In [17]:
for e in range(n_epochs):
    v_to_s = v_to_s.train()
    s_to_v = s_to_v.train()
    
    for i, sample in enumerate(trainloader):
        optimizer.zero_grad()
        
        batch_visual = sample['image_embedding'].cuda().float()
        batch_semantic = sample['class_embedding'].cuda().float()

        batch_classes = sample['class_id'].cuda() - 1
        
        e_hat = v_to_s(s_to_v(all_class_embeddings))
        delta = (e_hat - all_class_embeddings)
        surjection_loss = (delta * delta).sum(dim=-1).mean()
        
        s_out = v_to_s(batch_visual)
        
        same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
        same_class = same_class.detach()

        d_matrix = dist_matrix(s_out, all_class_embeddings)

        closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
        furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)
        
        l2_loss = (s_out * s_out).sum(dim=-1).mean()

        loss = positive_part(furthest_positive - closest_negative + margin)
        loss = alpha1 * loss.mean() + alpha2 * surjection_loss + alpha3 * l2_loss
        
        loss.backward()
        
#         c_hat = d_matrix.argmin(dim = -1)
#         print((c_hat == batch_classes).float().mean().item())
        
        optimizer.step()
        
        print(loss.item(), end=', ')
        
    if (e+1) % 50 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.7
    
    if (e+1) % 5 == 0:
        print('\n\n- Evaluation on epoch {}'.format(e))
        
        avg_accuracy = 0.
        avg_loss = 0.
        n = 0
        
        v_to_s = v_to_s.eval() 
        s_to_v = s_to_v.eval() 
        
        with torch.no_grad():
            for i, sample in enumerate(testloader):
                n += 1
                
                batch_visual = sample['image_embedding'].cuda().float()
                batch_semantic = sample['class_embedding'].cuda().float()

                batch_classes = sample['class_id'].cuda() - 1

                s_out = v_to_s(batch_visual)

                same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
                same_class = same_class.detach()

                d_matrix = dist_matrix(s_out, all_class_embeddings) 
                
                c_hat = (d_matrix + (1 - query_mask).float() * 1e6).argmin(dim = -1)

                closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
                furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)

                loss = positive_part(furthest_positive - closest_negative + margin)
                loss = alpha1 * furthest_positive.mean()

                avg_loss += loss.item()
                avg_accuracy += (c_hat == batch_classes).float().mean().item()

        avg_accuracy /= n
        avg_loss /= n

        print('Average acc.: {}, Average loss:{}\n\n'.format(avg_accuracy, avg_loss))


11.166008949279785, 11.16985034942627, 11.1709566116333, 11.179472923278809, 11.156553268432617, 11.176736831665039, 11.162323951721191, 11.162064552307129, 11.173615455627441, 11.173018455505371, 11.17451000213623, 11.170242309570312, 11.177309036254883, 11.151769638061523, 11.172592163085938, 11.16130542755127, 11.165106773376465, 11.151607513427734, 11.156949043273926, 11.162507057189941, 11.16007137298584, 11.164521217346191, 11.16877269744873, 11.166194915771484, 11.148180961608887, 11.17044734954834, 11.152498245239258, 11.160164833068848, 11.158604621887207, 11.158815383911133, 11.162564277648926, 11.146380424499512, 11.152549743652344, 11.157347679138184, 11.160743713378906, 11.163321495056152, 11.160299301147461, 11.153938293457031, 11.16557502746582, 11.147576332092285, 11.147515296936035, 11.15133285522461, 11.156330108642578, 11.153288841247559, 11.154268264770508, 11.164199829101562, 11.16197681427002, 11.15051555633545, 11.143962860107422, 11.163228034973145, 11.145195007

11.039643287658691, 11.057225227355957, 

KeyboardInterrupt: 

# Best results

with words only: 34.6%

with words+attributes:

# With attributes

In [1]:
import os
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt

from zsldataset import ZSLDataset
from models import ContinuousMap, ContinuousMapResidual

def dist_matrix(batch1, batch2):
    delta = batch1.unsqueeze(1) - batch2.unsqueeze(0)
    
    dist_matrix = (delta * delta).mean(dim=-1)
    
    return dist_matrix


def mag(u):
    return torch.dot(u, u)


def dist(u, v):
    return torch.dot(u - v, u - v)

In [2]:
from models import EncoderAttributes, DecoderAttributes

In [3]:
trainset = ZSLDataset('Data/APY/train', use_predicates=True, use_irevnet=True)
testset = ZSLDataset('Data/APY/test', use_predicates=True, use_irevnet=True)

In [4]:
bsize = 128
n_epochs = 200
num_classes = trainset.classes.shape[0]

dim_semantic = trainset[0]['class_embedding'].shape[0]
dim_visual = trainset[0]['image_embedding'].shape[0]
dim_attributes = trainset[0]['class_predicates'].shape[0]

all_class_embeddings = torch.tensor(np.array(trainset.class_embeddings)).cuda().float()
all_class_predicates = torch.tensor(np.array(trainset.class_predicates)).cuda().float()
classes_enum = torch.tensor(np.array(range(num_classes), dtype=np.int64)).cuda()

In [5]:
v_to_s = DecoderAttributes(dim_source=dim_visual, dim_target1=dim_attributes, dim_target2=dim_semantic, width=512).cuda()
s_to_v = EncoderAttributes(dim_source1=dim_semantic, dim_source2=dim_attributes, dim_target=dim_visual, width=512).cuda()

# optimizer = torch.optim.Adam(list(v_to_s.parameters()) + list(s_to_v.parameters()),
#                                 lr = 1e-3,
#                                 betas=(0.9, 0.999),
#                                 weight_decay=3e-2)

optimizer = torch.optim.SGD(list(v_to_s.parameters()) + list(s_to_v.parameters()),
                                lr = 1e-4,
                                momentum=.9,
                                weight_decay=3e-3,
                                nesterov=True)

In [6]:
mse = torch.nn.MSELoss()
triplet_loss = torch.nn.TripletMarginLoss(margin=1, p=2)
positive_part = torch.nn.ReLU()

In [7]:
trainloader = torch.utils.data.DataLoader(trainset, 
                                             batch_size=bsize, 
                                             shuffle=True, 
                                             num_workers=4, 
                                             pin_memory=True, 
                                             drop_last=True)

testloader = torch.utils.data.DataLoader(testset, 
                                         batch_size=bsize, 
                                         shuffle=True, 
                                         num_workers=4, 
                                         pin_memory=True, 
                                         drop_last=True)

In [8]:
gamma = 0.4

alpha1 = 10 # triplet
alpha2 = 1e-3 # surjection
alpha3 = 1e-3 # l2 loss

margin = 3

In [9]:
for e in range(n_epochs):
    v_to_s = v_to_s.train()
    s_to_v = s_to_v.train()
    
    for i, sample in enumerate(trainloader):
        optimizer.zero_grad()
        
        batch_visual = sample['image_embedding'].cuda().float()
        batch_semantic = sample['class_embedding'].cuda().float()

        batch_classes = sample['class_id'].cuda() - 1
         
        e_hat = v_to_s(s_to_v(all_class_embeddings, all_class_predicates))
        delta = (e_hat[1] - all_class_embeddings)
        surjection_loss = (delta * delta).sum(dim=-1).mean()
        delta = (e_hat[0] - all_class_predicates)
        surjection_loss = (1-gamma) * surjection_loss + gamma * (delta * delta).sum(dim=-1).mean()
        
        s_out = v_to_s(batch_visual)
        s_attr, s_word = s_out
        
        same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
        same_class = same_class.detach()

        d_matrix = (1 - gamma) * dist_matrix(s_word, all_class_embeddings) + \
                    gamma * dist_matrix(s_attr, all_class_predicates)
        
        closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
        furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)
        
        l2_loss = (1-gamma) * (s_word * s_word).sum(dim=-1).mean() + \
                    gamma * (s_attr * s_attr).sum(dim=-1).mean()

        loss = positive_part(furthest_positive - closest_negative + margin)
        loss = alpha1 * loss.mean() + alpha2 * surjection_loss + alpha3 * l2_loss
        
        loss.backward()
        
#         c_hat = d_matrix.argmin(dim = -1)
#         print((c_hat == batch_classes).float().mean().item())
        
        optimizer.step()
        
        print(loss.item(), end=', ')
        
    if (e+1) % 70 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.7
    
    if (e+1) % 5 == 0:
        print('\n\n- Evaluation on epoch {}'.format(e))
        
        avg_accuracy = 0.
        avg_loss = 0.
        n = 0
        
        v_to_s = v_to_s.eval() 
        s_to_v = s_to_v.eval() 
        
        with torch.no_grad():
            for i, sample in enumerate(testloader):
                n += 1
                
                batch_visual = sample['image_embedding'].cuda().float()
                batch_semantic = sample['class_embedding'].cuda().float()

                batch_classes = sample['class_id'].cuda() - 1

                s_out = v_to_s(batch_visual)
                s_attr, s_word = s_out

                same_class = classes_enum.unsqueeze(0) == batch_classes.unsqueeze(1)
                same_class = same_class.detach()

                d_matrix = (1 - gamma) * dist_matrix(s_word, all_class_embeddings) + \
                                gamma * dist_matrix(s_attr, all_class_predicates)
                
                c_hat = (d_matrix + (1 - query_mask).float() * 1e9).argmin(dim = -1)

                closest_negative, _ = (d_matrix + same_class.float() * 1e6).min(dim=-1)
                furthest_positive, _ = (d_matrix * (1 - same_class.float())).max(dim=-1)

                loss = positive_part(furthest_positive - closest_negative + margin)
                loss = alpha1 * furthest_positive.mean()

                avg_loss += loss.item()
                avg_accuracy += (c_hat == batch_classes).float().mean().item()

        avg_accuracy /= n
        avg_loss /= n

        print('Average acc.: {}, Average loss:{}\n\n'.format(avg_accuracy, avg_loss))




10.576552391052246, 10.571242332458496, 10.585657119750977, 10.570113182067871, 10.572693824768066, 10.564399719238281, 10.583442687988281, 10.563982009887695, 10.576849937438965, 10.566195487976074, 10.566062927246094, 10.572005271911621, 10.570043563842773, 10.579184532165527, 10.56896686553955, 10.578373908996582, 10.57557201385498, 10.574932098388672, 10.575716018676758, 10.579255104064941, 10.565694808959961, 10.569279670715332, 10.571930885314941, 10.576912879943848, 10.569809913635254, 10.577178001403809, 10.573882102966309, 10.56824779510498, 10.568777084350586, 10.569540023803711, 10.567770004272461, 10.564340591430664, 10.56863021850586, 10.567337036132812, 10.565375328063965, 10.571569442749023, 10.564568519592285, 10.549568176269531, 10.569628715515137, 10.561238288879395, 10.570425987243652, 10.567354202270508, 10.57348346710205, 10.555363655090332, 10.573796272277832, 10.557287216186523, 10.563777923583984, 10.557394981384277, 10.568008422851562, 10.55777359008789, 10.558

10.473814964294434, 10.472463607788086, 10.471417427062988, 10.47845458984375, 10.471652030944824, 10.477463722229004, 10.476791381835938, 10.465911865234375, 10.474359512329102, 10.47663688659668, 10.476705551147461, 10.47421646118164, 10.475882530212402, 10.474130630493164, 10.475974082946777, 10.466058731079102, 10.470616340637207, 10.476666450500488, 10.47673225402832, 10.468131065368652, 10.47529411315918, 10.473690032958984, 10.471055030822754, 10.477643966674805, 10.472545623779297, 10.476994514465332, 10.471497535705566, 10.475738525390625, 10.475276947021484, 10.477211952209473, 10.472213745117188, 10.477034568786621, 10.467114448547363, 10.470785140991211, 10.467985153198242, 10.469478607177734, 10.46585750579834, 10.468551635742188, 10.46728229522705, 10.474327087402344, 10.469099998474121, 10.47754192352295, 10.475008964538574, 10.472349166870117, 10.467121124267578, 10.469367027282715, 10.469293594360352, 10.469952583312988, 10.474010467529297, 10.463894844055176, 10.47112

10.421088218688965, 10.428439140319824, 10.421414375305176, 10.431675910949707, 10.426183700561523, 10.425251960754395, 10.42251968383789, 10.430203437805176, 10.427746772766113, 10.427931785583496, 10.424474716186523, 10.423203468322754, 10.421954154968262, 10.422307968139648, 10.42325496673584, 10.42492961883545, 10.42291259765625, 10.4251070022583, 10.42514705657959, 10.422247886657715, 10.4263277053833, 10.424420356750488, 10.421612739562988, 10.421426773071289, 10.420987129211426, 10.41860294342041, 10.419031143188477, 10.423933029174805, 10.42479133605957, 10.413914680480957, 10.430465698242188, 10.425638198852539, 10.422989845275879, 10.42755126953125, 10.422779083251953, 10.423589706420898, 10.42320442199707, 10.422731399536133, 10.415956497192383, 10.420620918273926, 10.418737411499023, 10.419844627380371, 10.420681953430176, 10.419843673706055, 10.422870635986328, 10.426002502441406, 10.426702499389648, 10.421138763427734, 10.420893669128418, 10.411648750305176, 10.4199552536

10.389780044555664, 10.388581275939941, 10.392325401306152, 10.385407447814941, 10.385101318359375, 10.387079238891602, 10.39152717590332, 10.383692741394043, 10.385562896728516, 10.387141227722168, 10.392574310302734, 10.3879976272583, 10.389190673828125, 10.384090423583984, 10.38803482055664, 10.386956214904785, 10.381670951843262, 10.386515617370605, 10.384057998657227, 10.385222434997559, 10.391252517700195, 10.39074420928955, 10.388833045959473, 10.38326358795166, 10.388726234436035, 10.382406234741211, 10.386628150939941, 10.3837890625, 10.38699722290039, 10.38428783416748, 10.385071754455566, 10.385074615478516, 10.384552955627441, 10.3930025100708, 10.38403606414795, 10.382936477661133, 10.384008407592773, 10.389664649963379, 10.388554573059082, 10.385189056396484, 10.387979507446289, 10.384016990661621, 10.384309768676758, 10.387726783752441, 10.388214111328125, 10.389104843139648, 10.390302658081055, 10.385220527648926, 10.388657569885254, 10.385799407958984, 10.3816080093383

10.356931686401367, 10.353549003601074, 10.356621742248535, 10.35800552368164, 10.361464500427246, 10.36059284210205, 10.359007835388184, 10.355916023254395, 10.357375144958496, 10.358688354492188, 10.360599517822266, 10.35999870300293, 10.357586860656738, 10.356282234191895, 10.357471466064453, 10.355326652526855, 10.356111526489258, 10.355250358581543, 10.361103057861328, 10.360528945922852, 10.360970497131348, 10.360697746276855, 10.360490798950195, 10.360000610351562, 10.359432220458984, 10.35892391204834, 10.358602523803711, 10.354757308959961, 10.352926254272461, 10.354562759399414, 10.362146377563477, 10.359034538269043, 10.363341331481934, 10.361105918884277, 10.354168891906738, 10.354211807250977, 10.362714767456055, 10.357831001281738, 10.360580444335938, 10.359580993652344, 10.35484790802002, 10.354098320007324, 10.356512069702148, 10.360666275024414, 10.35781478881836, 10.359169960021973, 10.356884956359863, 10.361504554748535, 10.358880043029785, 10.355039596557617, 10.351

...

10.228909492492676, 10.223947525024414, 10.224092483520508, 10.224543571472168, 10.223194122314453, 10.223017692565918, 10.224357604980469, 10.224209785461426, 10.222232818603516, 10.222922325134277, 10.220561981201172, 10.229923248291016, 10.221733093261719, 10.22302532196045, 10.222084045410156, 10.223438262939453, 10.228765487670898, 10.221206665039062, 10.22155475616455, 10.223406791687012, 10.22494888305664, 10.22741985321045, 10.22127628326416, 10.220389366149902, 10.223527908325195, 10.22177505493164, 10.226601600646973, 10.219860076904297, 10.221745491027832, 10.226664543151855, 10.219436645507812, 10.22415542602539, 10.22732925415039, 10.219372749328613, 10.218865394592285, 10.225308418273926, 10.223690032958984, 10.22006607055664, 10.218730926513672, 10.221365928649902, 10.220911979675293, 10.222477912902832, 10.217391014099121, 10.221051216125488, 10.225433349609375, 10.22262954711914, 10.219839096069336, 10.23325252532959, 10.220973014831543, 10.21810245513916, 10.225475311

10.21730899810791, 10.216856956481934, 10.218402862548828, 10.217668533325195, 10.21939754486084, 10.21433162689209, 10.220257759094238, 10.217941284179688, 10.218643188476562, 10.217313766479492, 10.220815658569336, 10.215497970581055, 10.215494155883789, 10.222734451293945, 10.220854759216309, 10.218816757202148, 10.216733932495117, 10.215934753417969, 10.219422340393066, 10.216740608215332, 10.217855453491211, 10.22150707244873, 10.221894264221191, 10.216927528381348, 10.218863487243652, 10.21996784210205, 10.215937614440918, 10.2210111618042, 10.21967601776123, 10.215180397033691, 10.218958854675293, 10.218128204345703, 10.217766761779785, 10.219337463378906, 10.217061042785645, 10.217667579650879, 10.221380233764648, 10.215574264526367, 10.216870307922363, 10.216899871826172, 10.213994979858398, 10.215292930603027, 10.21312427520752, 10.22116756439209, 10.218613624572754, 10.219717025756836, 10.219171524047852, 10.220040321350098, 10.219868659973145, 10.218411445617676, 10.2163238

10.217594146728516, 10.210453987121582, 10.209402084350586, 10.21205997467041, 10.214550018310547, 10.210954666137695, 10.213065147399902, 10.215622901916504, 10.211838722229004, 10.213516235351562, 10.217329978942871, 10.217350959777832, 10.214166641235352, 10.212203025817871, 10.211584091186523, 10.209515571594238, 10.212644577026367, 10.210111618041992, 10.214520454406738, 10.21146297454834, 10.216190338134766, 10.214681625366211, 10.213846206665039, 10.214966773986816, 10.214344024658203, 10.216261863708496, 10.216092109680176, 10.215261459350586, 10.210403442382812, 10.211742401123047, 10.21306037902832, 10.217960357666016, 10.217479705810547, 10.215717315673828, 10.210357666015625, 10.208675384521484, 10.219587326049805, 10.213189125061035, 10.214136123657227, 10.210221290588379, 10.2109375, 10.215510368347168, 10.207810401916504, 10.214205741882324, 10.215084075927734, 10.210325241088867, 10.20970344543457, 10.214872360229492, 10.210503578186035, 10.216033935546875, 10.216620445

10.207953453063965, 10.207883834838867, 10.207527160644531, 10.208725929260254, 10.207771301269531, 10.206258773803711, 10.208145141601562, 10.208159446716309, 10.207987785339355, 10.209508895874023, 10.20935344696045, 10.206942558288574, 10.206900596618652, 10.209486961364746, 10.205013275146484, 10.20931339263916, 10.207016944885254, 10.210288047790527, 10.207576751708984, 10.210003852844238, 10.208322525024414, 10.20926284790039, 10.206392288208008, 10.207622528076172, 10.207440376281738, 10.213780403137207, 10.20853328704834, 10.20726203918457, 10.207549095153809, 10.204448699951172, 10.20775318145752, 10.21550178527832, 10.209959983825684, 10.204283714294434, 10.20522689819336, 10.209728240966797, 10.2131986618042, 10.206775665283203, 10.207565307617188, 10.210044860839844, 10.21066665649414, 10.206098556518555, 10.209321022033691, 10.209357261657715, 10.21132755279541, 10.205766677856445, 10.207047462463379, 10.205667495727539, 10.205832481384277, 10.212810516357422, 10.212116241

10.205192565917969, 10.207000732421875, 10.202558517456055, 10.208416938781738, 10.205802917480469, 10.201812744140625, 10.202725410461426, 10.198649406433105, 10.203556060791016, 10.20697021484375, 10.203592300415039, 10.204753875732422, 10.2039213180542, 10.204141616821289, 10.204055786132812, 10.204879760742188, 10.205310821533203, 10.20627212524414, 10.207159042358398, 10.20808219909668, 10.200845718383789, 10.207062721252441, 10.203288078308105, 10.204103469848633, 10.20678424835205, 10.202285766601562, 10.20561408996582, 10.202993392944336, 10.202098846435547, 10.2027587890625, 10.201300621032715, 10.204837799072266, 10.201014518737793, 10.202437400817871, 10.208267211914062, 10.20390796661377, 10.203934669494629, 10.20421028137207, 10.202451705932617, 10.203739166259766, 10.206840515136719, 10.206335067749023, 10.209610939025879, 10.203271865844727, 10.197952270507812, 10.197551727294922, 10.203694343566895, 10.203010559082031, 10.204320907592773, 10.19848918914795, 10.199051856